In [13]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading


In [14]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [15]:
def scrape_links(page_number):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://jogja.antaranews.com/jogja-terkini/{page_number}"
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article',{"class": "simple-post simple-big clearfix"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [16]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 9 links from page 1
Total Links: 9


In [17]:
def scrape_url(url):
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1',{"class": "post-title"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "tags-wrapper"})
            if author_elem:
                author_text = author_elem.find('span')
                author_text = author_text.text.split(':')[1]
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('span', {"class": "article-date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            # category_elements = soup.find('ul', {"class": "breadcrumb"})
            # if category_elements:
            #     category_text = category_elements.find_all('li')
            #     category_text= category_text[2].get_text()
            # else:
            category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "post-content clearfix font17"})
            # print(body_elem)
            if body_elem:
                # content_elem = body_elem.find_all('br')
                # print(content_elem)
                content_text = body_elem.text
                # for p in content_elem:
                #     content_text += p.text.strip() + "\n"
                
                # if content_text.strip():
                #     content_text=content_text
                # else:
                #     content_text ="Content not found"
            else:
                content_text ="Content not found"

            results.append({'title': title_text,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'region':'jogja',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [18]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [19]:
df = pd.DataFrame(results)
print(len(results))
df.head(6)

9


,title,author,category,date,content,region,link
0,DKP DIY meningkatkan kompetensi nelayan sambut...,Luqman Hakim\n\n\t\t\t\t Editor,Category not found,"Minggu, 24 September 2023 22:39 WIB",\n Yogyakarta (ANTARA) - Dinas Kel...,jogja,https://jogja.antaranews.com/berita/636825/dkp...
1,Pemda DIY melarang ASN berkomentar dan membagi...,Luqman Hakim\n\n\t\t\t\t Editor,Category not found,"Rabu, 27 September 2023 21:37 WIB",\n Yogyakarta (ANTARA) - Pemerinta...,jogja,https://jogja.antaranews.com/berita/637380/pem...
2,Dispar DIY menyiapkan paket wisata Sumbu Filosofi,Luqman Hakim\n\n\t\t\t\t Editor,Category not found,"Senin, 25 September 2023 15:48 WIB",\n Yogyakarta (ANTARA) - Dinas Par...,jogja,https://jogja.antaranews.com/berita/636972/dis...
3,Disperindag DIY mengoptimalkan ekspor produk m...,Luqman Hakim\n\n\t\t\t\t Editor,Category not found,"Jumat, 22 September 2023 0:53 WIB",\n Yogyakarta (ANTARA) - Dinas Per...,jogja,https://jogja.antaranews.com/berita/636345/dis...
4,Pakar UGM sebut peran apoteker dalam program r...,Luqman Hakim\n\n\t\t\t\t Editor,Category not found,"Rabu, 27 September 2023 10:44 WIB",\n Yogyakarta (ANTARA) - Guru Besa...,jogja,https://jogja.antaranews.com/berita/637320/pak...
5,Pakar sebut bisnis benih hortikultura memiliki...,Luqman Hakim\n\n\t\t\t\t Editor,Category not found,"Sabtu, 23 September 2023 6:47 WIB",\n Yogyakarta (ANTARA) - Guru Besa...,jogja,https://jogja.antaranews.com/berita/636573/pak...
